In [ ]:
!pip install -U imbalanced-learn
# conda install -c conda-forge imbalanced-learn 

In [ ]:
!pip install tqdm

In [ ]:
!git clone https://github.com/KU-DIC/LG_time_series_day16.git #코랩 사용

# [모델 평가 지표]
## 데이터 불균형 처리

##### jupyter notebook 단축키

- ctrl+enter: 셀 실행   
- shift+enter: 셀 실행 및 다음 셀 이동   
- alt+enter: 셀 실행, 다음 셀 이동, 새로운 셀 생성
- a: 상단에 새로운 셀 만들기
- b: 하단에 새로운 셀 만들기
- dd: 셀 삭제(x: 셀 삭제)
- 함수 ( ) 안에서 shift+tab: arguments description. shift+tab+tab은 길게 볼 수 있도록

## 1. 모듈 불러오기

In [ ]:
''' 기본 모듈 '''
import numpy as np
import pandas as pd
import scipy as sp
import random

''' 결과 평가용 모듈 '''
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

''' 불균형 처리 모듈 '''
from imblearn.under_sampling import *
from imblearn.over_sampling import *
# from tsai.all import *
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc, rcParams

%matplotlib inline

## 2. Imbalanced Data roblem
> 정확도(accuracy)가 높아도 데이터 갯수가 적은 class의 재현율(recall-rate)이 급격히 작아지는 현상이 발생함.

In [ ]:
def classification_result(n0, n1, title=""):
    rv1 = sp.stats.multivariate_normal([-1, 0], [[1, 0], [0, 1]])
    rv2 = sp.stats.multivariate_normal([+1, 0], [[1, 0], [0, 1]])
    X0 = rv1.rvs(n0, random_state=0)
    X1 = rv2.rvs(n1, random_state=0)
    X = np.vstack([X0, X1])
    y = np.hstack([np.zeros(n0), np.ones(n1)])

    x1min = -4; x1max = 4
    x2min = -2; x2max = 2
    xx1 = np.linspace(x1min, x1max, 1000)
    xx2 = np.linspace(x2min, x2max, 1000)
    X1, X2 = np.meshgrid(xx1, xx2)

    plt.contour(X1, X2, rv1.pdf(np.dstack([X1, X2])), levels=[0.05], linestyles="dashed")
    plt.contour(X1, X2, rv2.pdf(np.dstack([X1, X2])), levels=[0.05], linestyles="dashed")

    model = SVC(kernel="linear", C=1e4, random_state=0).fit(X, y)
    Y = np.reshape(model.predict(np.array([X1.ravel(), X2.ravel()]).T), X1.shape)
    plt.scatter(X[y == 0, 0], X[y == 0, 1], marker='o', label="0 클래스")
    plt.scatter(X[y == 1, 0], X[y == 1, 1], marker='o', label="1 클래스")
    plt.contour(X1, X2, Y, colors='k', levels=[0.5])
    y_pred = model.predict(X)
    plt.xlim(-4, 4)
    plt.ylim(-3, 3)
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.title(title)
    
    return model, X, y, y_pred

plt.figure(figsize=(12,6))
plt.subplot(121)
model1, X1, y1, y_pred1 = classification_result(200, 200, "balanced data (5:5)")
plt.subplot(122)
model2, X2, y2, y_pred2 = classification_result(200, 20, "imbalanced data (9:1)")
plt.tight_layout()

plt.show()

In [ ]:
print(classification_report(y1, y_pred1))
print(classification_report(y2, y_pred2))

### Sampling Example

In [ ]:
n0 = 200; n1 = 20
rv1 = sp.stats.multivariate_normal([-1, 0], [[1, 0], [0, 1]])
rv2 = sp.stats.multivariate_normal([+1, 0], [[1, 0], [0, 1]])
X0 = rv1.rvs(n0, random_state=0)
X1 = rv2.rvs(n1, random_state=0)
X_imb = np.vstack([X0, X1])
y_imb = np.hstack([np.zeros(n0), np.ones(n1)])

x1min = -4; x1max = 4
x2min = -2; x2max = 2
xx1 = np.linspace(x1min, x1max, 1000)
xx2 = np.linspace(x2min, x2max, 1000)
X1, X2 = np.meshgrid(xx1, xx2)

def classification_result2(X, y, title=""):
    plt.contour(X1, X2, rv1.pdf(np.dstack([X1, X2])), levels=[0.05], linestyles="dashed")
    plt.contour(X1, X2, rv2.pdf(np.dstack([X1, X2])), levels=[0.05], linestyles="dashed")
    model = SVC(kernel="linear", C=1e4, random_state=0).fit(X, y)
    Y = np.reshape(model.predict(np.array([X1.ravel(), X2.ravel()]).T), X1.shape)
    plt.scatter(X[y == 0, 0], X[y == 0, 1], marker='o', label="0 클래스")
    plt.scatter(X[y == 1, 0], X[y == 1, 1], marker='o', label="1 클래스")
    plt.contour(X1, X2, Y, colors='k', levels=[0.5])
    y_pred = model.predict(X)
    plt.xlim(-4, 4)
    plt.ylim(-3, 3)
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.title(title)
    return model

Under or Over Sampling을 사용하여 데이터 비율을 맞추면 정밀도(precision)가 향상됨.

## 3. Under Sampling

### 3.1 Random Under-Sampler
>무작위로 데이터를 없애는 단순 샘플링

In [ ]:
X_samp, y_samp = RandomUnderSampler(random_state=0).fit_resample(X_imb, y_imb)

plt.subplot(121)
classification_result2(X_imb, y_imb)
plt.subplot(122)
model_samp = classification_result2(X_samp, y_samp)

In [ ]:
print(classification_report(y_imb, model_samp.predict(X_imb)))

### 3.2 Tomek's link method
>토멕링크(Tomek’s link)란 서로 다른 클래스에 속하는 한 쌍의 데이터  (x+,x−) 로 서로에게 더 가까운 다른 데이터가 존재하지 않는 것

>즉 클래스가 다른 두 데이터가 아주 가까이 붙어있으면 토멕링크가 됨

>토멕링크 방법은 이러한 토멕링크를 찾은 다음 그 중에서 다수 클래스에 속하는 데이터를 제외하는 방법으로 경계선을 다수 클래스쪽으로 밀어붙이는 효과가 있음

In [ ]:
X_samp, y_samp = TomekLinks().fit_resample(X_imb, y_imb)

plt.subplot(121)
classification_result2(X_imb, y_imb)
plt.subplot(122)
model_samp = classification_result2(X_samp, y_samp)

In [ ]:
print(classification_report(y_imb, model_samp.predict(X_imb)))

### 3.3 Condensed Nearest Neighbour
> CNN(Condensed Nearest Neighbour) 방법은 1-NN 모형으로 분류되지 않는 데이터만 남기는 방법 
 >> 선텍된 데이터 집합을 S 라고 하면, 
 
 >> 소수 클래스 데이터를 모두 S 에 포함시킴
 
 >> 다수 데이터 중에서 하나를 골라서 가장 가까운 데이터가 다수 클래스이면 포함시키지 않고 아니면  S 에 포함시킴
 
 >> 더이상 선택되는 데이터가 없을 때까지 2를 반복
    
> 이 방법을 사용하면 기존에 선택된 데이터와 가까이 있으면서 같은 클래스인 데이터는 선택되지 않기 때문에 다수 데이터의 경우 선택되는 비율이 적어짐

In [ ]:
X_samp, y_samp = CondensedNearestNeighbour(random_state=0).fit_resample(X_imb, y_imb)

plt.subplot(121)
classification_result2(X_imb, y_imb)
plt.subplot(122)
model_samp = classification_result2(X_samp, y_samp)

In [ ]:
print(classification_report(y_imb, model_samp.predict(X_imb)))

### 3.4 One Sided Selection
> One Sided Selection은 토맥링크 방법과 Condensed Nearest Neighbour 방법을 섞은 것

> 토맥링크 중 다수 클래스를 제외하고 나머지 데이터 중에서도 서로 붙어있는 다수 클래스 데이터는 1-NN 방법으로 제외

In [ ]:
X_samp, y_samp = OneSidedSelection(random_state=0).fit_resample(X_imb, y_imb)

plt.subplot(121)
classification_result2(X_imb, y_imb)
plt.subplot(122)
model_samp = classification_result2(X_samp, y_samp)

In [ ]:
print(classification_report(y_imb, model_samp.predict(X_imb)))

## 4. Over Sampling

### 4.1 RandomOverSampler
> Random Over Sampling은 소수 클래스의 데이터를 반복해서 넣는 것(replacement)

> 가중치를 증가시키는 것과 비슷

In [ ]:
X_samp, y_samp = RandomOverSampler(random_state=0).fit_resample(X_imb, y_imb)

plt.subplot(121)
classification_result2(X_imb, y_imb)
plt.subplot(122)
model_samp = classification_result2(X_samp, y_samp)

In [ ]:
print(classification_report(y_imb, model_samp.predict(X_imb)))

### 4.2 ADASYN
> ADASYN(Adaptive Synthetic Sampling) 방법은 소수 클래스 데이터와 그 데이터에서 가장 가까운 k개의 소수 클래스 데이터 중 무작위로 선택된 데이터 사이의 직선상에 가상의 소수 클래스 데이터를 만드는 방법

In [ ]:
X_samp, y_samp = ADASYN(random_state=0).fit_resample(X_imb, y_imb)

plt.subplot(121)
classification_result2(X_imb, y_imb)
plt.subplot(122)
model_samp = classification_result2(X_samp, y_samp)

In [ ]:
print(classification_report(y_imb, model_samp.predict(X_imb)))

### 4.3 SMOTE
> SMOTE(Synthetic Minority Over-sampling Technique) 방법도 ADASYN 방법처럼 데이터를 생성하지만 생성된 데이터를 무조건 소수 클래스라고 하지 않고 분류 모형에 따라 분류함

In [ ]:
X_samp, y_samp = SMOTE(random_state=4).fit_resample(X_imb, y_imb)

plt.subplot(121)
classification_result2(X_imb, y_imb)
plt.subplot(122)
model_samp = classification_result2(X_samp, y_samp)

In [ ]:
print(classification_report(y_imb, model_samp.predict(X_imb)))

## 5. Time Series Oversampling

### 데이터 불러오기


In [ ]:
train_data = pd.read_table('/content/LG_time_series_day16/data/Wafer_TRAIN.tsv', header=None)
test_data = pd.read_table('/content/LG_time_series_day16/data/Wafer_TEST.tsv', header=None)

# train_data = pd.read_table("./data/Wafer_TRAIN.tsv", header=None)
# test_data = pd.read_table("./data/Wafer_TEST.tsv", header=None)

train_X = np.array(train_data.iloc[:, 1:]) # 라벨 제외
test_X = np.array(test_data.iloc[:, 1:]) # 라벨 제외

train_y = np.array(train_data[0].apply(lambda x: 0 if x == 1 else 1)) # 라벨만
test_y = np.array(test_data[0].apply(lambda x : 0 if x == 1 else 1)) # 라벨만

train_X_max = np.max(train_X) # 데이터의 최대값
train_X_min = np.min(train_X) # 데이터의 최소값

# min-max normalization
train_X = 2. * (train_X - train_X_min) / (train_X_max - train_X_min) - 1. 
test_X = 2. * (test_X - train_X_min) / (train_X_max - train_X_min) - 1.

# (data size, data length) -> (data size, data length, data dimension)
train_X = train_X.reshape(np.shape(train_X)[0], np.shape(train_X)[1], -1)

### 정상 데이터

In [ ]:
l = np.where(train_y == 0)[0]
np.random.shuffle(l)
for i in l[:3]:
    fig = plt.figure(figsize=(10, 4))
    fig = plt.plot(train_X[i],color='grey')
    fig = plt.title(f"Data Index : {i}, Class : Normal")
    fig = plt.xticks([])
    plt.show()

### 비정상 데이터

In [ ]:
l = np.where(train_y == 1)[0]
np.random.shuffle(l)
for i in l[:3]:
    fig = plt.figure(figsize=(10,4))
    fig = plt.plot(train_X[i],color='grey')
    fig = plt.title(f"Data Index : {i}, Class : Anomaly")
    fig = plt.xticks([])
    plt.show()

In [ ]:
""" 시각화 """

def plot1d(x, x2, ylim=(-1, 1)):
    plt.figure(figsize=(6, 3))
    steps = np.arange(x.shape[0])
    plt.plot(steps, x)
    plt.plot(steps, x2)
    plt.xlim(0, x.shape[0])
    plt.ylim(ylim)
    plt.tight_layout()
    plt.show()
    return

In [ ]:
""" Augmentation """

class Augmentation:
    def jitter(x, sigma=0.03):
        # 시계열에 noise를 추가합니다.
        return x + np.random.normal(loc=0., scale=sigma, size=x.shape)

    def scaling(x, sigma=0.1):
        # 시계열을 무작위 상수만큼 움직입니다.
        factor = np.random.normal(loc=1., scale=sigma, size=(x.shape[0],x.shape[2]))
        return np.multiply(x, factor[:,np.newaxis,:])

    def rotation(x):
        # 시계열을 무작위로 회전시킵니다.
        flip = np.random.choice([-1, 1], size=(x.shape[0],x.shape[2]))
        rotate_axis = np.arange(x.shape[2])
        np.random.shuffle(rotate_axis)    
        return flip[:,np.newaxis,:] * x[:,:,rotate_axis]

    def permutation(x, max_segments=5, seg_mode="equal"):
        # 시계열을 부분으로 나누고, 뒤섞습니다.
        orig_steps = np.arange(x.shape[1])

        num_segs = np.random.randint(1, max_segments, size=(x.shape[0]))

        ret = np.zeros_like(x)
        for i, pat in enumerate(x):
            if num_segs[i] > 1:
                if seg_mode == "random":
                    split_points = np.random.choice(x.shape[1]-2, num_segs[i]-1, replace=False)
                    split_points.sort()
                    splits = np.split(orig_steps, split_points)
                else:
                    splits = np.array_split(orig_steps, num_segs[i])
                warp = np.concatenate(np.random.permutation(splits)).ravel()
                ret[i] = pat[warp]
            else:
                ret[i] = pat
        return ret

    def magnitude_warp(x, sigma=0.2, knot=4):
        # 설정된 수의 매듭으로 큐빅 스플라인에 의해 생성된 곡선으로 곱합니다.
        from scipy.interpolate import CubicSpline
        orig_steps = np.arange(x.shape[1])

        random_warps = np.random.normal(loc=1.0, scale=sigma, size=(x.shape[0], knot+2, x.shape[2]))
        warp_steps = (np.ones((x.shape[2],1))*(np.linspace(0, x.shape[1]-1., num=knot+2))).T
        ret = np.zeros_like(x)
        for i, pat in enumerate(x):
            warper = np.array([CubicSpline(warp_steps[:,dim], random_warps[i,:,dim])(orig_steps) for dim in range(x.shape[2])]).T
            ret[i] = pat * warper

        return ret

In [ ]:
plot1d(train_X[0], Augmentation.jitter(train_X)[0])

In [ ]:
plot1d(train_X[0], Augmentation.scaling(train_X)[0])

In [ ]:
plot1d(train_X[0], Augmentation.rotation(train_X)[0])

In [ ]:
plot1d(train_X[0], Augmentation.magnitude_warp(train_X)[0])

### Augmentation 된 데이터셋 만들기

In [ ]:
train_X_jitter = Augmentation.jitter(train_X)
train_X_scailing = Augmentation.scaling(train_X)
train_X_rotation = Augmentation.rotation(train_X)
train_X_permutation = Augmentation.permutation(train_X)
train_X_magnitude_warp = Augmentation.magnitude_warp(train_X)

In [ ]:
aug_train_X = np.concatenate([train_X, train_X_jitter, train_X_scailing, train_X_rotation, train_X_permutation, train_X_magnitude_warp])

In [ ]:
print("original data size : " + str(train_X.shape))
print("augmented data size : " + str(aug_train_X.shape))

---